In [1]:
import json
import yaml
from mstrio.connection import Connection
#from mstr_robotics.prepare_AI_data import parse_json
from mstr_robotics.mstr_classes import mstr_global
#from mstr_robotics.json_checksum_handler import json_checksum_handler
#from mstr_robotics.read_out_prj_obj import read_gen
from mstr_robotics.redis_db import  redis_bi_analysis,redis_mstr_json
#from mstr_robotics._helper import msic
from mstr_robotics._connectors import mstr_api
from mstr_robotics.prepare_AI_data import export_mstr_md



i_mstr_global=mstr_global()
#=msic()
#i_read_gen=read_gen()
#i_parse_json=parse_json()
#i_json_checksum_handler=json_checksum_handler()
#i_mstr_to_json=mstr_to_json()
i_mstr_api=mstr_api()
i_redis_mstr_json=redis_mstr_json()
i_export_mstr_md=export_mstr_md()


with open('..\\config\\mstr_redis_y.yml', 'r') as openfile:
    mstr_redis_y = yaml.safe_load(openfile)


#redis_mstr_d={"project_prefix":{"B7CA92F04B9FAE8D941C3E9B7E0CD754":"mstr_dev",
#                                "F8AD06364B29E4FFA3AEC9A34E706A47":"mstr_test"}}
#redis_mstr_d={"project_prefix":{"F8AD06364B29E4FFA3AEC9A34E706A47":"mstr_test"}}


In [ ]:
redis_con_d=mstr_redis_y["redis_env_d"]["redis_dev"]
project_prefix=mstr_redis_y["project_prefix"]
prefix_map=mstr_redis_y["prefix_map"]
searches_used_in_prp_d_l=mstr_redis_y["searches_used_in_prp_d_l"]


## Connect to MSTR & Redis

In [ ]:
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]
conn = Connection(**conn_params)


i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)


## Save objects to redis

### Daily Search Update

In [ ]:

err_d_l=[]
for pre in project_prefix:
    env_prefix=project_prefix[pre]
    conn.select_project(pre)
    search_result = i_mstr_api.run_mstr_search(conn=conn,
                        search_id="96648F2B492150A6AA27DDB3744E32B4")
    print(search_result)

    if search_result["totalItems"] > 0:
        all_obj_d_l=search_result["result"]
        """
        err_d_l.append(i_redis_mstr_json.save_obj_json_to_redis(i_redis_bi_analysis=i_redis_bi_analysis                                                    
                                                        ,conn=conn
                                                        ,prefix_map=prefix_map
                                                        , all_obj_d_l=all_obj_d_l
                                                        , env_prefix=env_prefix)
                        )
        print(str(all_obj_d_l) + " uploaded to " + env_prefix)
        """
all_obj_d_l

### Full load

In [ ]:
# Initialize Redis connection
env_prefix="mstr_test"
i_redis_bi_analysis.emergency_flush_db(confirm_phrase='FLUSH_ALL_DATA')

for pre in project_prefix:
    env_prefix=project_prefix[pre]
    conn.select_project(pre)
    all_obj_d_l=i_export_mstr_md.read_out_prj_by_type(conn)

    err_d_l=i_redis_mstr_json.save_obj_json_to_redis(i_redis_bi_analysis=i_redis_bi_analysis
                                                    ,conn=conn
                                                    ,prefix_map=prefix_map
                                                    , all_obj_d_l=all_obj_d_l
                                                    , env_prefix=env_prefix)

    i_redis_mstr_json.run_searches_redis(conn=conn
                                        ,i_redis_bi_analysis=i_redis_bi_analysis
                                        ,prefix_map=prefix_map
                                        ,env_prefix=env_prefix
                                        ,searches_used_in_prp_d_l=searches_used_in_prp_d_l)
    print("finished read: " + pre)


### Shortcut Folder

In [10]:
folder_id="B70AF11248C00F7D4339079310F8B3F1"
project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn.select_project(project_id)
all_obj_d_l=i_mstr_global.get_obj_from_sh_fold(conn,folder_id=folder_id)

Folder object named: 'testObjs' with ID: 'B70AF11248C00F7D4339079310F8B3F1'


### Single Objects

In [7]:
all_obj_d_l=[]
all_obj_d_l.append({"id":"C58A830B4DEB35BF601D119134952B83","type":"3","subtype":"768"})
#all_obj_d_l.append({"id":"C58A830B4DEB35BF601D119134952B83","type":"3","subtype":"768"})

In [11]:
redis_mstr_d={"project_prefix":{"B7CA92F04B9FAE8D941C3E9B7E0CD754":"mstr_dev",
                                "74F9B2164869627AF637FDA5D4A121B3":"mstr_test"}}
env_prefix="mstr_test"
project_id="74F9B2164869627AF637FDA5D4A121B3"
#project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
env_prefix=redis_mstr_d["project_prefix"][project_id]
env_prefix
#object_id_l=["98EB31B54122FFB738E6E08A2F29421A","A90DC09E4A4EFF57D57734B418EDB667"]
#all_obj_d_l=[]
#all_obj_d_l.append({"id":"862780DC499A14D74FEC7EB2EF317DA2","type":"15","subtype":"3840"})


conn.select_project(project_id)
err_d_l=i_redis_mstr_json.save_obj_json_to_redis(i_redis_bi_analysis=i_redis_bi_analysis                                                    
                                                    ,conn=conn
                                                    ,prefix_map=prefix_map
                                                    , all_obj_d_l=all_obj_d_l
                                                    , env_prefix=env_prefix)

err_d_l

Uploaded object definition for E6A7C8EE4BB767E8380E3A896092C0E6
Uploaded object definition for BA38E83548881BF83570CCABE2CB9B4B
Uploaded object definition for 8827902011D3EB22C000B4B2D86C964F
Uploaded object definition for 8827901011D3EB22C000B4B2D86C964F
Uploaded object definition for E2DBE7164E4389A7500778B94BEEBCB7
Uploaded object definition for 4C051F3511D3E877C000B3B2D86C964F
Uploaded object definition for 7A4ABE9111D5AC6FC000D98A4CC5F24F
Uploaded object definition for 8D679D3C11D3E4981000E787EC6DE8A4
Error preparing objefdfddct definition: 'tree'
Uploaded object definition for 5D55D4504907E73F560BFBA95AD22A66
Uploaded object definition for FF7BB69911D501F0C00051916B98494F
Uploaded object definition for 7C0CC9C641A360A6AEC1D8AFD23225B5
Uploaded object definition for 6F4FF0C84F271FC9851E9F8F502FF1C0
Uploaded object definition for 98EB31B54122FFB738E6E08A2F29421A
Uploaded object definition for 381980B211D40BC8C000C8906B98494F
Uploaded object definition for 1C0A3D964E21D5D517D915958D

[]

In [ ]:
i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)
obj_def_d=i_redis_bi_analysis.fetch_key_value("mstr_test:REPORT_DEFINITION:C58A830B4DEB35BF601D119134952B83")
obj_comp_def_d=i_redis_bi_analysis.fetch_key_value("mstr_test:REPORT_DEFINITION:C58A830B4DEB35BF601D119134952B83")


In [ ]:
file_name=obj_comp_def_d["value"]["information"]["objectId"] +"_"
file_name+=obj_comp_def_d["value"]["information"]["subType"]
file_name+=obj_comp_def_d["value"]["information"]["name"]

In [ ]:
import os
json_l=[obj_comp_def_d,obj_def_d]
i=0
for file in json_l:
    export_path="C:\\coding\\Python_environments\\mstr_robotics\\export"
    file_name=str(i)
    i+=1
    file_name+=obj_comp_def_d["value"]["information"]["objectId"] +"_"
    file_name+=obj_comp_def_d["value"]["information"]["subType"]
    file_name+=obj_comp_def_d["value"]["information"]["name"]
    filename_1 = os.path.join(export_path, f"{file_name}.json")
    with open(filename_1, 'w', encoding='utf-8') as f:
        json.dump(file, f, indent=2, ensure_ascii=False)
    

In [ ]:
i=0
child_l=obj_def_d["value"]["dataSource"]["filter"]["tree"]["children"]
while i < len(child_l):
    child_l[i]["filt_obj"]=child_l[i]["predicateTree"]
    i+=1
obj_def_d["value"]["dataSource"]["filter"]["tree"]["children"]=child_l
obj_def_d